In [1]:
import numpy as np
import pandas as pd
from googleapiclient.discovery import build

In [2]:
# Youtube Data API:
yt = build(
    serviceName = "youtube",
    version = "v3",
    developerKey = "" # my key
)

In [73]:
###### Channels data

# Channels titles:
channel_title = ["Tom Scott"]

# Channels Ids (from content = "https://www.youtube.com/channel/):
channel_id = ["UCBa659QWEk1AI4Tg--mrJ2A"]

# Choose the channel:
c = 0

### Extract the data from all the videos of the channel

# Content details:
content = yt.channels().list(
    id = channel_id[c],
    part = "contentDetails"
).execute()

# Upload Id:
upload_id = content["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

# All the channel's videos:
all_videos = []
next_pg_token = None
res = yt.playlistItems().list(
    playlistId = upload_id,
    maxResults = 50,
    part = "snippet",
    pageToken = next_pg_token
).execute()
while True:
    res = yt.playlistItems().list(
        playlistId = upload_id,
        maxResults = 50,
        part = "snippet",
        pageToken = next_pg_token
    ).execute()
    all_videos += res["items"]
    next_pg_token = res.get("nextPageToken")
    if next_pg_token is None:
        break

# Videos statistics:
videos_ids = list(map(lambda x: x["snippet"]["resourceId"]["videoId"], all_videos))
stats = []
for i in range(0, len(videos_ids), 40):
    res = (yt).videos().list(
        id = ",".join(videos_ids[i:i + 40]),
        part = "statistics"
    ).execute()
    stats += res["items"]

# Put it in a dataframe:
video_id, views, likes, dislikes, favorites, comments = [], [], [], [], [], []
for i in range(len(stats)):
    video_id += [stats[i]["id"]]
    stats_i = stats[i]["statistics"]
    views += [stats_i[k] if "viewCount" in stats_i.keys() else None for k in ["viewCount"]]
    likes += [stats_i[k] if "likeCount" in stats_i.keys() else None for k in ["likeCount"]]
    dislikes += [stats_i[k] if "dislikeCount" in stats_i.keys() else None for k in ["dislikeCount"]]
    favorites += [stats_i[k] if "favoriteCount" in stats_i.keys() else None for k in ["favoriteCount"]]
    comments += [stats_i[k] if "commentCount" in stats_i.keys() else None for k in ["commentCount"]]
df_videos = pd.DataFrame(
    {
        "channel_title": channel_title[c],
        "channel_id": channel_id[c],
        "video_title": list(map(lambda x: x["snippet"]["title"], all_videos)),
        "video_id": video_id,
        "video_upload_date": list(map(lambda x: x["snippet"]["publishedAt"], all_videos)),
        "views": views,
        "likes": likes,
        "dislikes": dislikes,
        "favorites": favorites,
        "comments": comments        
    }
)


In [74]:
df_videos

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,favorites,comments
0,Tom Scott,UCBa659QWEk1AI4Tg--mrJ2A,The world's only float-through McDonalds,A6F96xSoLPg,2021-10-08T16:43:23Z,1970655,126218,890,0,4496
1,Tom Scott,UCBa659QWEk1AI4Tg--mrJ2A,I thought the Schmid Peoplemover was impossible,A2g4u9F9i90,2021-09-30T16:02:45Z,2599002,142264,802,0,5598
2,Tom Scott,UCBa659QWEk1AI4Tg--mrJ2A,The world's most useful model railway,6TLcaJdsRr0,2021-09-24T14:00:54Z,1940432,106995,619,0,3180
3,Tom Scott,UCBa659QWEk1AI4Tg--mrJ2A,The public toll road with no speed limit,10Y-gWNJ2Sw,2021-09-17T08:11:31Z,1933711,128726,813,0,3952
4,Tom Scott,UCBa659QWEk1AI4Tg--mrJ2A,Three strange river crossings,KVXVNX4Crh4,2021-09-05T13:07:18Z,1466991,87171,428,0,1908
...,...,...,...,...,...,...,...,...,...,...
623,Tom Scott,UCBa659QWEk1AI4Tg--mrJ2A,On A Pirate Ship - Jay Foreman feat. Mad Cap'n...,dK-pMy_jOKI,2007-09-03T10:43:18Z,79529,2557,32,0,211
624,Tom Scott,UCBa659QWEk1AI4Tg--mrJ2A,Fun With A Giant Slingshot,dNOWXi8KoOs,2007-04-08T13:49:52Z,131017,1551,84,0,227
625,Tom Scott,UCBa659QWEk1AI4Tg--mrJ2A,"Pastaaargh: pasta, cooked with a kettle",sZ363XfT8fk,2006-07-05T12:03:03Z,52002,964,17,0,186
626,Tom Scott,UCBa659QWEk1AI4Tg--mrJ2A,Extra Light Pancakes,nkTyWTC9UZc,2006-07-04T12:40:49Z,63502,1458,16,0,193
